In [ ]:
import spacy.cli

spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,'./drive/MyDrive/MedicineDetection/data')

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin
import spacy
import sys
import data3 as dt
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [ ]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
BASE_DIR = "./drive/MyDrive/MedicineDetection/"

In [ ]:
trainData = dt.train
print(trainData)

[('I took 30 ml of A Class syrup as prescribed by my doctor to relieve my cough and congestion.', {'entities': [(7, 12, 'quantity'), (16, 29, 'medicine')]}), ('I mixed 30 ml of A Class syrup with a glass of water and drank it before bed to help me sleep.', {'entities': [(8, 13, 'quantity'), (17, 30, 'medicine')]}), ('The pharmacist recommended that I take 30 ml of A Clas syrup three times a day to treat my cold symptoms.', {'entities': [(39, 44, 'quantity'), (47, 60, 'medicine')]}), ('I added 30 ml of A Clas syrup to a bottle of water and shook it up to create a refreshing and medicinal drink.', {'entities': [(8, 13, 'quantity'), (17, 29, 'medicine')]}), ("I measured out 30 ml of A Clas syrup and mixed it with some juice to make it more palatable for my child's medicine.", {'entities': [(15, 20, 'quantity'), (24, 37, 'medicine')]}), ('The doctor prescribed an A Artivid 150mg injection to help control my chronic pain.', {'entities': [(35, 40, 'quantity'), (25, 34, 'medicine')]}), ('I ha

In [ ]:
db = DocBin()


for text, annot in tqdm(trainData):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk(os.path.join(BASE_DIR,"./train.spacy"))

100%|██████████| 54/54 [00:00<00:00, 977.50it/s]


In [ ]:
!python -m spacy init fill-config ./drive/MyDrive/MedicineDetection/config/base_config.cfg ./drive/MyDrive/MedicineDetection/config/config.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-30 20:24:19.167263: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
drive/MyDrive/MedicineDetection/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train ./drive/MyDrive/MedicineDetection/config/config.cfg --output ./drive/MyDrive/MedicineDetection/output --paths.train ./drive/MyDrive/MedicineDetection/train.spacy --paths.dev ./drive/MyDrive/MedicineDetection/train.spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-30 18:44:41.468309: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory:
drive/MyDrive/MedicineDetection/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-30 18:44:42,483] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-30 18:44:42,496] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-30 18:44:42,500] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-30 18:44:47,240] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0x39024000 @  0x7f82e32bf2a4 0x7f82d7bffe09 0x7f82d7bfecdf 0x7f82d7bfb675 0x7f82d7bfbe2e 0x4f750a 0x4

In [ ]:
nlp1 = spacy.load(r"./drive/MyDrive/MedicineDetection/output/model-best")

In [ ]:
colors = {'quantity': "#85C1E9", 'medicine':'#AFE1AF'}
options = {"ents": ['quantity','medicine'], "colors": colors}

In [ ]:
text = "I appreciate that the A C Ford Kid tablets come in a convenient 200 mg dosage that is easy for my child to swallow."
doc = nlp1(text)
spacy.displacy.render(doc, style="ent", jupyter=True, options=options)

In [ ]:
text = "5 strips of Crocin tablet, 10 strips of Dolo 650mg tablets and 5 bottles of glucose solution"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent", jupyter=True, options=options)

In [ ]:
text = "50 boxes Heart Injection"
doc = nlp1(text)
spacy.displacy.render(doc, style="ent", jupyter=True, options=options)

In [ ]:
examples = [
    ("Gallab 400mg tablets is an antibiotic used to treat a variety of bacterial infections.",{"entities":[(7,12,"quantity"),(0,6,"medicine")]}),
("It is important to take Gallab 400mg tablets exactly as prescribed by a healthcare provider and to finish the entire course of treatment to ensure the best possible outcome.",{"entities":[(31,36,"quantity"),(24,30,"medicine")]}),
("Clonapik 2mg tablets are a prescription medication used to treat anxiety disorders, panic disorders, and certain types of seizure disorders.",{"entities":[(8,13,"quantity"),(0,8,"medicine")]}),
(" Clonapik tablets contain 2mg of the active ingredient clonazepam, which belongs to a class of drugs called benzodiazepines.",{"entities":[(26,30,"quantity"),(1,10,"medicine")]}),
("Floxi 200mg tablets are a prescription medication used to treat infections caused by bacteria.",{"entities":[(6,11,"quantity"),(0,5,"medicine")]}),
("Floxi tablets contain 200mg of the active ingredient ciprofloxacin, which belongs to a class of drugs called quinolone antibiotics.",{"entities":[(22,27,"quantity"),(0,13,"medicine")]}),
("Flucold Clear Syrup is a over-the-counter (OTC) medicinal syrup used to treat symptoms of the common cold and flu.",{"entities":[(0,19,"medicine")]}),
("Flucold Clear Syrup contains a combination of active ingredients including paracetamol, phenylephrine, and chlorpheniramine, which work together to reduce fever, relieve body aches, and decongest the nasal passages.",{"entities":[(0,20,"medicine")]}),
("1% Itom Dusting Powder is a topical medication used to treat fungal infections of the skin, such as athlete's foot and ringworm",{"entities":[(0,2,"quantity"),(3,22,"medicine")]}),
("Itom Dusting Powder contains 1% of the active ingredient miconazole, which is an antifungal agent that works by stopping the growth of fungi.",{"entities":[(29,33,"quantity"),(1,21,"medicine")]}),
("Itrakrypt 100 Capsules are a prescription medication used to treat infections caused by bacteria.",{"entities":[(10,14,"quantity"),(0,10,"medicine")]}),
("Itrakrypt capsule contains 100mg of the active ingredient amikacin, which is an aminoglycoside antibiotic that works by inhibiting the growth of bacteria. ",{"entities":[(27,32,"quantity"),(0,18,"medicine")]}),
]

In [ ]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annot in examples:
        pred = ner_model(input_)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        example.append(temp)
    scores = scorer.score(example)
    return scores

results = evaluate(nlp1, examples)
for key, value in results.items():
    print("{:<10} {:<10}".format(str(key), str(value)))

token_acc  1.0       
token_p    1.0       
token_r    1.0       
token_f    1.0       
sents_p    None      
sents_r    None      
sents_f    None      
tag_acc    None      
pos_acc    None      
morph_acc  None      
morph_micro_p None      
morph_micro_r None      
morph_micro_f None      
morph_per_feat None      
dep_uas    None      
dep_las    None      
dep_las_per_type None      
ents_p     None      
ents_r     None      
ents_f     None      
ents_per_type None      
cats_score 0.0       
cats_score_desc macro F   
cats_micro_p 0.0       
cats_micro_r 0.0       
cats_micro_f 0.0       
cats_macro_p 0.0       
cats_macro_r 0.0       
cats_macro_f 0.0       
cats_macro_auc 0.0       
cats_f_per_type {}        
cats_auc_per_type {}        
